# Practico Mentoria - Analisis Exploratorio y Curación de Datos

---

### Importaciones

In [86]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

In [87]:
# Set seed for reproducibility
np.random.seed(0)

In [88]:
player_df = pd.read_csv('../Datasets/football_player.csv')
team_df = pd.read_csv('../Datasets/football_team.csv')
match_df = pd.read_csv('../Datasets/football_match.csv', parse_dates=['date'])

In [89]:
print("Shape 'player_df' = {}".format(player_df.shape))
print("Shape 'team_df'   = {}".format(team_df.shape))
print("Shape 'match_df'  = {}".format(match_df.shape))

Shape 'player_df' = (11060, 40)
Shape 'team_df'   = (288, 22)
Shape 'match_df'  = (25979, 12)


---

# 1. Importacion de los datos

Calculemos el rango de fechas de los partidos

In [90]:
match_df['date'].max() - match_df['date'].min()

Timedelta('2868 days 00:00:00')

Indiquemos que la columna `date` es una fechas como indica la [documentación](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#datetime-handling)

# 2. Etiquetas de variables/columnas: no usar caracteres especiales

Chequar que no haya caracteres fuera de `a-Z`, `0-9` y `_` en los nombres de columnas de los Dataframes:
* `player_df`
* `team_df`
* `match_df`

In [91]:
import re

dataset_list = [player_df, team_df, match_df]
#for i in range(0, len(dataset_list)):
for dataset in dataset_list:
    print("-"*50)
    #print("Index: ", i)
    #dataset = dataset_list[i]
    for col_name in list(dataset):
        match = re.findall('([^a-zA-Z0-9_]+)', col_name)
        if len(match) > 0:
            print(col_name, "\t-->\tUnsupported characters: ", match)
        else:
            print(col_name)

--------------------------------------------------
player name 	-->	Unsupported characters:  [' ']
birthday
height_m
weight_kg
overall_rating
potential
preferred foot 	-->	Unsupported characters:  [' ']
crossing
finishing
heading accuracy 	-->	Unsupported characters:  [' ']
short passing 	-->	Unsupported characters:  [' ']
volleys
dribbling
curve
free kick accuracy 	-->	Unsupported characters:  [' ', ' ']
long passing 	-->	Unsupported characters:  [' ']
ball control 	-->	Unsupported characters:  [' ']
acceleration
sprint speed 	-->	Unsupported characters:  [' ']
agility
reactions
balance
shot power 	-->	Unsupported characters:  [' ']
jumping
stamina
strength
long shots 	-->	Unsupported characters:  [' ']
aggression
interceptions
positioning
vision
penalties
marking
standing tackle 	-->	Unsupported characters:  [' ']
sliding tackle 	-->	Unsupported characters:  [' ']
gk_diving
gk_handling
gk_kicking
gk_positioning
gk_reflexes
--------------------------------------------------
team long 

# 3. Agregar nuevas caracteristicas

Agregar al Dataframe `player_df` una nueva columna que sea `imc` correspondiente al **Indice de Masa Corporal**

Link:
* https://www.texasheart.org/heart-health/heart-information-center/topics/calculadora-del-indice-de-masa-corporal-imc/

In [53]:
def imc_calculator(weights, heights):
    imc = []
    for i in np.arange(0, len(weights)):
        imc.append(float("{0:.2f}".format(weights[i]/(heights[i]*heights[i]))), )

    return imc

player_df = player_df.assign(imc=pd.Series(imc_calculator(player_df.weight_kg, player_df.height_m)).values)
player_df.head(5)

,player name,birthday,height_m,weight_kg,overall_rating,potential,preferred foot,crossing,finishing,heading accuracy,...,penalties,marking,standing tackle,sliding tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,imc
0,Aaron Appindangoye,1992-02-29,1.83,84.82,63.60,67.60,right,48.60,43.60,70.60,...,47.60,63.80,66.00,67.80,5.60,10.60,9.60,7.60,7.60,25.33
1,Aaron Cresswell,1989-12-15,1.70,66.22,66.97,74.48,left,70.79,49.45,52.94,...,53.12,69.39,68.79,71.52,12.18,8.67,14.24,10.36,12.91,22.91
2,Aaron Doran,1991-05-13,1.70,73.94,67.00,74.19,right,68.12,57.92,58.69,...,60.54,22.04,21.12,21.35,14.04,11.81,17.73,10.12,13.50,25.58
3,Aaron Galindo,1982-05-08,1.83,89.81,69.09,70.78,right,57.22,26.26,69.26,...,41.74,70.61,70.65,68.04,14.17,11.17,22.87,11.17,10.17,26.82
4,Aaron Hughes,1979-11-08,1.83,69.85,73.24,74.68,right,45.08,38.84,73.04,...,52.96,77.60,76.04,74.60,8.28,8.32,24.92,12.84,11.92,20.86


# 4. Tratar valores faltantes

Veamos cuantos valores nulos tenemos

In [54]:
player_missing_values_count = player_df.isnull().sum()

player_missing_values_count[player_missing_values_count > 0]

volleys           478
curve             478
agility           478
balance           478
jumping           478
vision            478
sliding tackle    478
dtype: int64

In [55]:
team_missing_values_count = team_df.isnull().sum()

team_missing_values_count[team_missing_values_count > 0]

Series([], dtype: int64)

In [56]:
match_missing_values_count = match_df.isnull().sum()

match_missing_values_count[match_missing_values_count > 0]

Series([], dtype: int64)

Algunas tecnicas para tratar los _missing values_:
* **Eliminar** muestras o variables que tienen datos faltantes.
* **Imputar** los valores perdidos, es decir, sustituirlos por estimaciones por ejemplo la `media`, la `moda` ó usando `KNN`.

A) Analizar si es conveniente **Eliminar** las muestras o variables con datos faltantes del Dataframe `player_df`. 

B) Aplicar la **Imputacion** usando la `media` o `moda` sobre las columnas con _missing values_ del Dataframe `player_df`.

**Hint**:
* Para la imputacion usando la `media`, `moda` ver el siguiente link:  
    https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html

### ¿Eliminar los _missing values_? Justificar

Yo no eliminaria los missing values en este caso porque eso implicaria eliminar 500 filas de nuestro dataset, solo porque estan faltando 7 features de las 40 que son en total, por lo que estariamos eliminando mucha informacion útil junto con ellas.

### Imputacion usando Media y Moda

In [63]:
# Rellenando usando moda ya que tiene mas sentido utilizar los valores mas usuales a que el promedio
player_df.fillna(player_df.mode(), inplace=True)

# 5. Normalizacion de columnas

Primero que todo la notación:
* $x = [x_1, x_2, ..., x_n]$
* $\mu$: Media
* $\sigma$: Desviacion Estandar

Ahora normalizaremos algunas de las columnas del Dataframe, para ello usaremos dos tipos de normalizacion:

* Min-Max:  
$$
z_i = \frac{x_i - min(x)}{max(x) - min(x)}
$$
* Z-score
$$
z_i = \frac{x_i - \mu}{\sigma}
$$

Normalizar la columna `crossing` usando **Min-Max**.

Normalizar la columna `short_passing` usando **Z-score**.

**Hints**:
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html

### Min-Max

In [74]:
print(player_df.crossing.head(10))

scaler = preprocessing.MinMaxScaler()
player_df[["crossing"]] = scaler.fit_transform(player_df[["crossing"]])

print(player_df.crossing.head(10))

0    48.60
1    70.79
2    68.12
3    57.22
4    45.08
5    73.89
6    47.57
7    78.04
8    12.00
9    63.89
Name: crossing, dtype: float64
0    0.511036
1    0.777231
2    0.745202
3    0.614443
4    0.468810
5    0.814419
6    0.498680
7    0.864203
8    0.071977
9    0.694458
Name: crossing, dtype: float64


### Z-score

In [79]:
print(player_df["short passing"].head(10))

scaler = preprocessing.MinMaxScaler()
player_df[["short passing"]] = sp.stats.zscore(player_df[["short passing"]])

print(player_df["short passing"].head(10))

0    60.60
1    62.27
2    65.12
3    64.70
4    64.76
5    78.26
6    63.57
7    76.27
8    23.00
9    68.95
Name: short passing, dtype: float64
0    0.017238
1    0.140868
2    0.351853
3    0.320761
4    0.325202
5    1.324605
6    0.237107
7    1.177285
8   -2.766282
9    0.635387
Name: short passing, dtype: float64


# 6. Codificar variables

> Las variables categóricas deben ser etiquetadas como variables numéricas, no como cadenas.

Codificar la variable `country_name` del Dataframe `match_df`

In [92]:
print(set(match_df["country name"]))

le = preprocessing.LabelEncoder()
match_df[["country name"]] = le.fit_transform(match_df[["country name"]])

print(set(match_df["country name"]))

{'Portugal', 'Belgium', 'Italy', 'Spain', 'France', 'England', 'Scotland', 'Switzerland', 'Germany', 'Netherlands', 'Poland'}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


More about preprocessing in:
* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing